In [20]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String, DateTime
from sqlalchemy.sql import func

from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session
from sqlalchemy import select, create_engine, Column

from sqlalchemy.dialects.postgresql import JSONB

import json

In [31]:
class Base(DeclarativeBase):
    pass



class Chip(Base):
    __tablename__ = "chip"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    created_by: Mapped[str] = mapped_column(String(30))
    created = mapped_column(DateTime(timezone=True), server_default=func.now())
    time_updated = mapped_column(DateTime(timezone=True), onupdate=func.now())
    notes: Mapped[Optional[str]] = mapped_column(String(30))
    gds_link: Mapped[Optional[str]] = mapped_column(String(30))
    image_url: Mapped[Optional[str]] = mapped_column(String(30))

    arrays: Mapped[List["Array"]] = relationship(
         back_populates="chip", cascade="all, delete-orphan"
    )

    def __repr__(self) -> str:
        return f"Chip(id={self.id!r}, name={self.name!r})"
    

class Array(Base):
    __tablename__ = "array"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    chip_id: Mapped[int] = mapped_column(ForeignKey("chip.id"))
    
    created = mapped_column(DateTime(timezone=True), server_default=func.now())
    time_updated = mapped_column(DateTime(timezone=True), onupdate=func.now())

    chip: Mapped["Chip"] = relationship(back_populates="arrays")

    devices: Mapped[List["Device"]] = relationship(
         back_populates="array", cascade="all, delete-orphan"
    )
    

    def __repr__(self) -> str:
        return f"Array(id={self.id!r}, name={self.name!r})"
    

class Device(Base):
    __tablename__ = "device"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    array_id: Mapped[int] = mapped_column(ForeignKey("array.id"))

    created_by: Mapped[str] = mapped_column(String(30))
    created = mapped_column(DateTime(timezone=True), server_default=func.now())
    time_updated = mapped_column(DateTime(timezone=True), onupdate=func.now())
    notes: Mapped[Optional[str]] 

    array: Mapped["Array"] = relationship(back_populates="devices")

    targets: Mapped[List["Target"]] = relationship(
         back_populates="device", cascade="all, delete-orphan"
    )
    

    def __repr__(self) -> str:
        return f"Device(id={self.id!r}, name={self.name!r}, array_id={self.array_id})"
    
## Think about postgresql
class Target(Base):
    __tablename__ = "target"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    #metadata = Column(JSONB, nullable=True)
    device_id: Mapped[int] = mapped_column(ForeignKey("device.id"))

    
    created = mapped_column(DateTime(timezone=True), server_default=func.now())
    time_updated = mapped_column(DateTime(timezone=True), onupdate=func.now())
    

    device: Mapped["Device"] = relationship(back_populates="targets")
    

    def __repr__(self) -> str:
        return f"Device(id={self.id!r}, name={self.name!r}, array_id={self.array_id})"

In [32]:
# Retrieve value from .env file
from decouple import config
from sqlalchemy import URL


HOST= config("HOST")
PORT= config("PORT")
DBNAME= config("DBNAME")
USER= config("USER")
PASSWORD= config("PASSWORD")

use_aws = False

if use_aws:
    url_object = URL.create(
        "mysql+pymysql",
        username=USER,
        password=PASSWORD,  # plain (unescaped) text
        host=HOST,
        database=DBNAME,
    )
else:
    url_object = "sqlite+pysqlite:///:memory:"



from sqlalchemy import create_engine
engine = create_engine(url_object, echo=True)
Base.metadata.create_all(engine)


2023-02-25 23:43:25,852 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-25 23:43:25,853 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("chip")
2023-02-25 23:43:25,853 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-25 23:43:25,854 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("chip")
2023-02-25 23:43:25,854 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-25 23:43:25,855 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("array")
2023-02-25 23:43:25,856 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-25 23:43:25,856 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("array")
2023-02-25 23:43:25,856 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-25 23:43:25,857 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("device")
2023-02-25 23:43:25,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-25 23:43:25,858 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("device")
2023-02-25 23:43:25,858 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-25 

In [8]:


with Session(engine) as session:

    chip_1 = Chip(
        name="20_09_B",
        devices=[Device(name="A_a_c")],
    )
   
    session.add_all([chip_1])

    session.commit()


In [16]:
from sqlalchemy import select





stmt = select(Device).where(Device.name == "A_a_c")
target_device = session.scalars(stmt).one()

In [17]:
target_device

Device(id=1, name='A_a_c')

In [21]:
with Session(engine) as session:

    odmr = Experiment(
        name="ODMR"
    )

    laserscan = Experiment(
        name="LASERSCAN"
    )
   
    session.add_all([odmr, laserscan])

    session.commit()

In [24]:
stmt = select(Experiment).where(Experiment.name == "ODMR")
experiment = session.scalars(stmt).first()


In [25]:
target_device.experiments = [experiment]

In [26]:
session.commit()

/tmp/ipykernel_17188/631814250.py:1: SAWarning: Object of type <Device> not in session, add operation along 'Experiment.device' won't proceed
  session.commit()


In [28]:
target_device.experiments

[Experiment(id=1, name='ODMR')]